# **Tarea 3 - Word Embeddings 📚**

**Integrantes:**

**Fecha límite de entrega 📆:** 3 de mayo.

**Tiempo estimado de dedicación:**


**Instrucciones:**
- El ejercicio consiste en:
    - Responder preguntas relativas a los contenidos vistos en los vídeos y slides de las clases. 
    - Entrenar Word2Vec y Word Context Matrix sobre un pequeño corpus.
    - Evaluar los embeddings obtenidos en una tarea de clasificación.
- La tarea se realiza en grupos de **máximo** 2 personas. Puede ser invidivual pero no es recomendable.
- La entrega es a través de u-cursos a más tardar el día estipulado arriba. No se aceptan atrasos.
- El formato de entrega es este mismo Jupyter Notebook.
- Al momento de la revisión tu código será ejecutado. Por favor verifica que tu entrega no tenga errores de compilación. 
- En el horario de auxiliar pueden realizar consultas acerca de la tarea a través del canal de Discord del curso. 


**Referencias**

Vídeos: 

- [Linear Models](https://youtu.be/zhBxDsNLZEA)
- [Neural Networks](https://youtu.be/oHZHA8h2xN0)
- [Word Embeddings](https://youtu.be/wtwUsJMC9CA)

## **Preguntas teóricas 📕 (2 puntos).** ##
Para estas preguntas no es necesario implementar código, pero pueden utilizar pseudo código.

### **Parte 1: Modelos Lineales (1 ptos)**

Suponga que tiene un dataset de 10.000 documentos etiquetados por 4 categorías: política, deporte, negocios y otros. 

**Pregunta 1**: Diseñe un modelo lineal capaz de clasificar un documento según estas categorías donde el output sea un vector con una distribución de probabilidad con la pertenencia a cada clase. 

Especifique: representación de los documentos de entrada, parámetros del modelo, transformaciones necesarias para obtener la probabilidad de cada etiqueta y función de pérdida escogida. **(0.5 puntos)**

**Respuesta**: Representación escogida del documento de entrada: Bag of words por ejemplo

Parámetros del modelo: Matriz de pesos W en donde la columna 1 tengan mayor importancia las palabras de política, columna 2 sobre deporte, columna 3 sobre negocios y columna 4 el resto

Transformaciones necesarias: Aplicar softmax al vector de output del modelo.

Función de pérdida escogida: Cross-entropy es el ideal, pero pueden escoger cualquier función de perdida multiclase que alcanze el mínimo cuando las predicciones son correctas

**Pregunta 2**: Explique cómo funciona el proceso de entrenamiento en este tipo de modelos y su evaluación. **(0.5 puntos)**

**Respuesta**: El objetivo del entrenamiento es minimizar la loss del modelo. Se van tuneando los parámetros de la matriz W hasta encontrar uno que minimice la loss del modelo. La evaluación se hace sobre datos que no se han observado para comprobar generalización del modelo. 

### **Parte 2: Redes Neuronales (1 ptos)** 

Supongamos que tenemos la siguiente red neuronal.

![image.png](https://drive.google.com/uc?export=view&id=1fFTjtMvH6MY8o42_vj010y8eTuCVb5a3)

**Pregunta 1**: En clases les explicaron como se puede representar una red neuronal de una y dos capas de manera matemática. Dada la red neuronal anterior, defina la salida $\vec{\hat{y}}$ en función del vector $\vec{x}$, pesos $W^i$, bias $b^i$ y funciones $g,f,h$. 

Adicionalmente liste y explicite las dimensiones de cada matriz y vector involucrado en la red neuronal. **(0.5 Puntos)**

**Respuesta**: 

Formula:
$\vec{\hat{y}} = NN_{MLP3}(\vec{x}) =$

Dimensiones: 

- $\vec{x}$: 3
- $W¹$: 3x2
- $\vec{b}¹$: 2
- $W²$: 2x3
- $\vec{b}²$: 3
- $W³:$ 3x1
- $\vec{b}³:$ 1
- $W⁴:$ 1x4
- $\vec{b}⁴:$ 4

**Pregunta 2**: Explique qué es backpropagation. ¿Cuales serían los parámetros a evaluar en la red neuronal anterior durante backpropagation? **(0.25 puntos)**

**Respuesta**: "Backpropagation es una técnica eficiente para evaluar el gradiente de una loss function L en una red neuronal feed-forward con respecto a todos sus parámetros" (Cita de clases). Los parámetros serian de $W¹,\vec{b}¹$ a $W⁴,\vec{b}⁴$

**Pregunta 3**: Explique los pasos de backpropagation. En la red neuronal anterior: Cuales son las derivadas que debemos calcular para poder obtener $\vec{\delta^l_{[j]}}$ en todas las capas? **(0.25 puntos)**

**Respuesta**: Los 4 pasos son:

- Aplicar el vector x y propagarlo por toda la red
- Evaluar $\delta$ para todas las unidades ocultas
- Propagar los $\delta$ desde el final al inicio de la red
- Ocupar la formula de $\frac{\partial L}{\partial W}$

En la red anterior se necesitan las derivadas: $f', g',h'$

## **Preguntas prácticas 💻 (4 puntos).** ##

### **Parte 3: Word Embeddings**

En la auxiliar 2 se nombraron dos formas de crear word vectors:

-  Distributional Vectors.
-  Distributed Vectors.

El objetivo de esta parte es comparar las dos embeddings obtenidos de estas dos estrategias en una tarea de clasificación.

In [ ]:
import re  
import pandas as pd 
from time import time  
from collections import defaultdict 
import string 
import multiprocessing
import os
import gensim
import sklearn
from sklearn import linear_model
from collections import Counter
import numpy as np
import scipy
import nltk
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report

# word2vec
from gensim.models import Word2Vec, KeyedVectors, FastText
from gensim.models.phrases import Phrases, Phraser
from sklearn.model_selection import train_test_split
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

#### **Parte A (1 punto)** 

En esta parte debe crear una matriz palabra contexto, para esto, complete el siguiente template:

```python
class WordContextMatrix:

  def __init__(self, vocab_size):
    # se sugiere agregar un una estructura de datos para guardar las
    # palabras del vocab y para guardar el conteo de coocurrencia
    ...
    
  def add_word_to_vocab(self, word):
    ...
  
  def build_matrix(self):
    ...

  def matrix2dict(self):
    # se recomienda transformar la matrix a un diccionario de embedding.
    ...

```

puede modificar los parámetros o métodos si lo considera necesario. Para probar la matrix puede utilizar el siguiente corpus.

```python
corpus = [
  "I like deep learning.",
  "I like NLP.",
  "I enjoy flying."
]
```

Obteniendo una matriz parecia a esta:

***Resultado esperado***: 

| counts   | I  | like | enjoy | deep | learning | NLP | flying | . |   
|----------|---:|-----:|------:|-----:|---------:|----:|-------:|--:|
| I        | 0  |  2   |  1    |    0 |  0       |   0 | 0      | 0|            
| like     |  2 |    0 |  0    |    1 |  0       |   1 | 0      | 0 | 
| enjoy    |  1 |    0 |  0    |    0 |  0       |   0 | 1      | 0 |
| deep     |  0 |    1 |  0    |    0 |  1       |   0 | 0      | 0 |  
| learning |  0 |    0 |  0    |    1 |  0       |   0 | 0      | 1 |          
| NLP      |  0 |    1 |  0    |    0 |  0       |   0 | 0      | 1 |
| flying   |  0 |    0 |  1    |    0 |  0       |   0 | 0      | 1 | 
| .        |  0 |    0 |  0    |    0 |  1       |   1 | 1      | 0 | 

``

**Respuesta:**

In [1]:
import scipy
import numpy as np


class WordContextMatrix:

  def __init__(self, vocab_size, window_size, dataset, tokenizer):
    self.vocab_size = vocab_size
    self.window_size = window_size
    self.tokenizer = tokenizer
    self.dataset = dataset
    self.word2index = {}
    self.size = 0
    self.coor_matrix = scipy.sparse.lil_matrix((vocab_size + 1, vocab_size))

    self.add_word_to_vocab('unk')
    
  def add_word_to_vocab(self, word):
    if self.size < self.vocab_size and word not in self.word2index:
      self.word2index[word] = self.size
      self.size += 1

  def build_matrix(self):
    for text in self.dataset:
      tokens = self.tokenizer(text)
      for token in tokens:
        self.add_word_to_vocab(token)
      for token in tokens:
        ind_word = tokens.index(token)
        contexts = get_contexts(ind_word, self.window_size, tokens)
        for context in contexts:
          if token in self.word2index and context in self.word2index:
            ind_focus_word = self.word2index[token]
            ind_cont_word = self.word2index[context]
            self.coor_matrix[ind_focus_word, ind_cont_word] += 1.0
          elif token not in self.word2index and context in self.word2index:
            ind_cont_word = self.word2index[context]
            self.coor_matrix[0, ind_cont_word] += 1.0

  def matrix2dict(self):
    embeddings = {}
    for word, idx in self.word2index.items():
      embeddings[word] = self.coor_matrix[idx, :].toarray()[0]
    return embeddings

def get_contexts(ind_word, w_size, tokens):
    slice_start = ind_word - w_size if (ind_word - w_size >= 0) else 0
    slice_end = len(tokens) if (ind_word + w_size + 1 >= len(tokens)) else ind_word + w_size + 1
    first_part = tokens[slice_start: ind_word]
    last_part = tokens[ind_word + 1: slice_end]
    contexts = tuple(first_part + last_part)
    return contexts

In [ ]:
corpus = [
  "I like deep learning.",
  "I like NLP.",
  "I enjoy flying."
]



In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
wcm = WordContextMatrix(7, 1, corpus, word_tokenize)
wcm.build_matrix()

In [ ]:
wcm.coor_matrix.toarray()

array([[0., 1., 0., 0., 0., 1., 0.],
       [0., 0., 2., 0., 0., 0., 0.],
       [0., 2., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
wcm.matrix2dict()

{'.': array([0., 0., 0., 0., 1., 0., 1.]),
 'I': array([0., 0., 2., 0., 0., 0., 0.]),
 'NLP': array([0., 0., 1., 0., 0., 1., 0.]),
 'deep': array([0., 0., 1., 0., 1., 0., 0.]),
 'learning': array([0., 0., 0., 1., 0., 1., 0.]),
 'like': array([0., 2., 0., 1., 0., 0., 1.]),
 'unk': array([0., 1., 0., 0., 0., 1., 0.])}

#### **Parte B (1.5 puntos)**

En esta parte es debe entrenar Word2Vec de gensim y construir la matriz palabra contexto utilizando el dataset de diálogos de los Simpson. 

Utilizando el dataset adjunto con la tarea:

In [ ]:
data_file = "drive/MyDrive/dialogue-lines-of-the-simpsons.zip"
df = pd.read_csv(data_file)
stopwords = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/english.txt'
).values
stopwords = Counter(stopwords.flatten().tolist())
df = df.dropna().reset_index(drop=True) # Quitar filas vacias

In [ ]:
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [ ]:
punctuation = string.punctuation + "«»“”‘’…—"
def simple_tokenizer(doc, lower=False):
    if lower:
        tokenized_doc = doc.translate(str.maketrans(
            '', '', punctuation)).lower().split()

    tokenized_doc = doc.translate(str.maketrans('', '', punctuation)).split()

    tokenized_doc = [
        token for token in tokenized_doc if token.lower() not in stopwords
    ]
    return tokenized_doc
content = df['spoken_words']
sentences = [simple_tokenizer(doc) for doc in content.values]

**Pregunta 1**: Ayudándose de los pasos vistos en la auxiliar, entrene los modelos Word2Vec. **(0.75 punto)** (Hint, le puede servir explorar un poco los datos)

**Respuesta**:

In [ ]:
w2v_model = Word2Vec(min_count=10,
                      window=4,
                      size=200,
                      sample=6e-5,
                      alpha=0.03,
                      min_alpha=0.0007,
                      negative=20,
                      workers=multiprocessing.cpu_count())

In [ ]:
w2v_model.build_vocab(sentences, progress_per=10000)

2022-04-21 22:36:44,774 : INFO : collecting all words and their counts
2022-04-21 22:36:44,778 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-04-21 22:36:44,802 : INFO : PROGRESS: at sentence #10000, processed 33320 words, keeping 10655 word types
2022-04-21 22:36:44,836 : INFO : PROGRESS: at sentence #20000, processed 67371 words, keeping 17128 word types
2022-04-21 22:36:44,869 : INFO : PROGRESS: at sentence #30000, processed 104569 words, keeping 23061 word types
2022-04-21 22:36:44,900 : INFO : PROGRESS: at sentence #40000, processed 138266 words, keeping 27246 word types
2022-04-21 22:36:44,929 : INFO : PROGRESS: at sentence #50000, processed 170239 words, keeping 31169 word types
2022-04-21 22:36:44,958 : INFO : PROGRESS: at sentence #60000, processed 200105 words, keeping 34573 word types
2022-04-21 22:36:44,988 : INFO : PROGRESS: at sentence #70000, processed 233512 words, keeping 38228 word types
2022-04-21 22:36:45,067 : INFO : PROGRESS: at se

In [ ]:
t = time()
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=15, report_delay=10)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
if not os.path.exists('./pretrained_models'):
    os.mkdir('./pretrained_models')
w2v_model.save('./pretrained_models/w2v.model')

2022-04-21 22:36:48,922 : INFO : training model with 2 workers on 6492 vocabulary and 200 features, using sg=0 hs=0 sample=6e-05 negative=20 window=4
2022-04-21 22:36:50,002 : INFO : EPOCH 1 - PROGRESS: at 40.24% examples, 77362 words/s, in_qsize 3, out_qsize 0
2022-04-21 22:36:51,318 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-21 22:36:51,342 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-04-21 22:36:51,349 : INFO : EPOCH - 1 : training on 444264 raw words (202914 effective words) took 2.4s, 84389 effective words/s
2022-04-21 22:36:52,376 : INFO : EPOCH 2 - PROGRESS: at 45.47% examples, 89940 words/s, in_qsize 3, out_qsize 0
2022-04-21 22:36:53,473 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-04-21 22:36:53,503 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-04-21 22:36:53,509 : INFO : EPOCH - 2 : training on 444264 raw words (203257 effective words) took 2.1s, 94689 effective wo

Time to train the model: 0.62 mins


2022-04-21 22:37:26,498 : INFO : saved ./pretrained_models/w2v.model


**Pregunta 2**: Cree una matriz palabra contexto usando el mismo dataset. Configure el largo del vocabulario 7028. Puede que esto tarde un poco. **(0.75 punto)** 

**Respuesta:**

In [ ]:
wcm = WordContextMatrix(1000, 4, content.values, word_tokenize)

In [ ]:
wcm.build_matrix()

In [ ]:
embs = wcm.matrix2dict()

#### **Parte C (1.5 puntos): Aplicar embeddings para clasificar**

Ahora utilizaremos los embeddings que acabamos de calcular para clasificar palabras basadas en su polaridad (positivas o negativas). 

Para esto ocuparemos el lexicón AFINN incluido en la tarea, que incluye una lista de palabras y un 1 si su connotación es positiva y un -1 si es negativa.

In [ ]:
AFINN = 'drive/MyDrive/AFINN_full.csv'
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)

Hint: Para w2v y la wcm son esperables KeyErrors debido a que no todas las palabras del corpus de los simpsons tendrán una representación en AFINN. Para el caso de la matriz palabra contexto se recomienda convertir su matrix a un diccionario. Pueden utilizar esta función auxiliar para filtrar las filas en el dataframe que no tienen embeddings (como w2v no tiene token UNK se deben ignorar).

In [ ]:
def try_apply(model,word):
    try:
        aux = model[word]
        return True
    except KeyError:
        #logger.error('Word {} not in dictionary'.format(word))
        return False

**Pregunta 1**: Transforme las palabras del corpus de AFINN a la representación en embedding que acabamos de calcular (con ambos modelos). 

Su dataframe final debe ser del estilo [embedding, sentimiento], donde los embeddings corresponden a $X$ y el sentimiento asociado con el embedding a $y$ (positivo/negativo, 1/-1). 

Para ambos modelos, separar train y test de acuerdo a la siguiente función. **(0.5 puntos)**

```python 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)
```

**Respuesta**:

- Word Context Matrix

In [ ]:
df_afinn = df_afinn[df_afinn[0].apply(lambda x: try_apply(embs,x))]
df_afinn[0] = df_afinn[0].apply(lambda x: embs[x])
X = np.stack(df_afinn[0].values)
y = df_afinn[1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


**Pregunta 2**: Entrenar una regresión logística (vista en auxiliar) y reportar accuracy, precision, recall, f1 y confusion_matrix para ambos modelos. Por qué se obtienen estos resultados? Cómo los mejorarías? Como podrías mejorar los resultados de la matriz palabra contexto? es equivalente al modelo word2vec? **(1 punto)**

**Respuesta**:

In [ ]:
reg = linear_model.LogisticRegression(penalty='l2', solver='liblinear', C=1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf = confusion_matrix(y_test, y_pred)

In [ ]:
logger.info("The accuracy is {0}".format(acc))
logger.info("The precision is {0}".format(pre))
logger.info("The recall is {0}".format(rec))
logger.info("The f1 score is {0}".format(f1))
logger.info("The confusion matrix:\n{0}".format(conf))

2022-04-21 22:39:18,382 : INFO : The accuracy is 1.0
2022-04-21 22:39:18,385 : INFO : The precision is 1.0
2022-04-21 22:39:18,390 : INFO : The recall is 1.0
2022-04-21 22:39:18,394 : INFO : The f1 score is 1.0
2022-04-21 22:39:18,398 : INFO : The confusion matrix:
[[6 0]
 [0 5]]


In [ ]:
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)
df_afinn = df_afinn[df_afinn[0].apply(lambda x: try_apply(w2v_model,x))]
df_afinn[0] = df_afinn[0].apply(lambda x: w2v_model[x])
X = np.stack(df_afinn[0].values)
y = df_afinn[1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=y)
reg = linear_model.LogisticRegression(penalty='l2', solver='liblinear', C=1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf = confusion_matrix(y_test, y_pred)
logger.info("The accuracy is {0}".format(acc))
logger.info("The precision is {0}".format(pre))
logger.info("The recall is {0}".format(rec))
logger.info("The f1 score is {0}".format(f1))
logger.info("The confusion matrix:\n{0}".format(conf))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2022-04-21 22:39:30,129 : INFO : The accuracy is 0.5813953488372093
2022-04-21 22:39:30,130 : INFO : The precision is 0.0
2022-04-21 22:39:30,133 : INFO : The recall is 0.0
2022-04-21 22:39:30,135 : INFO : The f1 score is 0.0
2022-04-21 22:39:30,137 : INFO : The confusion matrix:
[[50  0]
 [36  0]]


Estos resultados son pésimos. Hay 2 razones posibles: El dataset es muy pequeño cómo para que estos modelos logren aprender bien las relaciones entre las palabras o puede que con los diálogos de los Simpson no se obtengan buenos embeddings para clasificar palabras por sentimiento. Se podría mejorar añadiendo mas datos, por ejemplo los subtítulos de las películas o simplemente buscar otro dataset más grande.

# Bonus: +0.25 puntos en cualquier pregunta

**Pregunta 1**: Replicar la parte anterior utilizando embeddings pre-entrenados en un dataset más grande y obtener mejores resultados. Les puede servir [ésta](https://radimrehurek.com/gensim/downloader.html#module-gensim.downloader) documentacion de gensim **(0.25 puntos)**.

**Respuesta**: